In [1]:
import pandas as pd

import sqlite3
conn = sqlite3.connect('./ipl_database.db')

# Create a cursor object
cursor = conn.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# Iterate over the table names
for table in tables:
    table_name = table[0]
    print(f"Table: {table_name}")
    sql_query = 'SELECT * FROM '+ table_name+';'
    df = pd.read_sql_query(sql_query, conn)
    xlsx_file_path = "C:/Users/anubr/Python Projects/IPL/assets_xls/"

    # Convert the DataFrame to a excel file
    df.to_excel(xlsx_file_path+table_name+".xlsx", index=False)

    # Print a message indicating the conversion is complete
    print(f"DataFrame has been successfully converted to CSV file: {xlsx_file_path+table_name}.xls")

    
conn.close()

Table: ipl_ball_by_ball
DataFrame has been successfully converted to CSV file: C:/Users/anubr/Python Projects/IPL/assets_xls/ipl_ball_by_ball.xls
Table: ipl_venue_list
DataFrame has been successfully converted to CSV file: C:/Users/anubr/Python Projects/IPL/assets_xls/ipl_venue_list.xls
Table: ipl_venue_innings_1
DataFrame has been successfully converted to CSV file: C:/Users/anubr/Python Projects/IPL/assets_xls/ipl_venue_innings_1.xls
Table: ipl_venue_innings_2
DataFrame has been successfully converted to CSV file: C:/Users/anubr/Python Projects/IPL/assets_xls/ipl_venue_innings_2.xls
Table: ipl_match_list
DataFrame has been successfully converted to CSV file: C:/Users/anubr/Python Projects/IPL/assets_xls/ipl_match_list.xls
Table: ipl_player_list
DataFrame has been successfully converted to CSV file: C:/Users/anubr/Python Projects/IPL/assets_xls/ipl_player_list.xls


In [11]:
import sqlite3
import data_manipulation
def get_batting_positions_id_innings(match_id, innings):
    conn = sqlite3.connect('./ipl_database.db')
    cursor = conn.cursor()

    # SQL query to fetch all unique batters in the specified match ID and innings
    query = '''
    SELECT DISTINCT batter
    FROM ipl_ball_by_ball
    WHERE ID = ? AND innings = ?
    ORDER BY overs, ballnumber
    '''

    cursor.execute(query, (match_id, innings))
    batting_positions = [row[0] for row in cursor.fetchall()]

    conn.close()
    return batting_positions

def get_all_players(match_id):
    conn = sqlite3.connect('./ipl_database.db')
    cursor = conn.cursor()

    # Retrieve the team players from the ipl_match_list table for the given match ID
    cursor.execute("SELECT Team1Players, Team2Players FROM ipl_match_list WHERE ID = ?;", (match_id,))
    team_players = cursor.fetchone()

    conn.close()

    if team_players:
        # Split the string to get the list of players for both teams
        team1_players = team_players[0][1:-1].split(", ")
        team2_players = team_players[1][1:-1].split(", ")
        all_players = team1_players + team2_players
        return all_players
    else:
        print("No match found for the given ID.")
        return None

def get_remaining_players(match_id, innings):
    conn = sqlite3.connect('./ipl_database.db')
    cursor = conn.cursor()

    # Retrieve toss information to determine which team is batting first
    cursor.execute("SELECT TossWinner, TossDecision FROM ipl_match_list WHERE ID = ?;", (match_id,))
    toss_data = cursor.fetchone()
    team1, team2 = None, None
    if toss_data:
        toss_winner, toss_decision = toss_data

        # Determine the team batting first based on toss decision
        if toss_decision.lower() == 'bat':
            batting_team = toss_winner
        else:
            # If the decision is to field, the other team is batting first
            cursor.execute("SELECT Team1Players, Team2Players FROM ipl_match_list WHERE ID = ?;", (match_id,))
            teams_data = cursor.fetchone()
            team1, team2 = teams_data
            batting_team = team1 if toss_winner == team2 else team2

        # Retrieve the players' names from the ipl_match_list table for the batting team
        cursor.execute("SELECT Team1Players, Team2Players FROM ipl_match_list WHERE ID = ?;", (match_id,))
        players_data = cursor.fetchone()

        conn.close()

        if players_data:
            team1_players_str = players_data[0] if batting_team == team1 else players_data[1]
            team2_players_str = players_data[1] if batting_team == team2 else players_data[0]

            # Convert the string format to a list of player names
            batting_team_players = eval(team1_players_str) if batting_team == team1 else eval(team2_players_str)

            # Get the batters who batted
            batters_list = data_manipulation.get_batting_positions_id_innings(match_id, innings)

            # Remove the batters who batted from the list of all players in the batting team
            remaining_players = [player for player in batting_team_players if player not in batters_list]

            return remaining_players
        else:
            print("No match found for the given ID.")
            return None
    else:
        print("No toss information found for the given ID.")
        return None

# Example usage:
match_id = 1312200
innings = 2
remaining_players = get_remaining_players(match_id, innings)
print("Remaining Players:", remaining_players)
print(len(remaining_players))

# Example usage:
match_id = 1312200
innings = 1
remaining_players = get_remaining_players(match_id, innings)
print("Remaining Players:", remaining_players)

Remaining Players: ['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D Padikkal', 'SO Hetmyer', 'R Ashwin', 'R Parag', 'TA Boult', 'OC McCoy', 'M Prasidh Krishna', 'YS Chahal']
11
Remaining Players: ['M Prasidh Krishna', 'YS Chahal']


In [59]:
import sqlite3
import ast
import data_manipulation

def remaining_players(ID, innings, batting_list):
    conn = sqlite3.connect('./ipl_database.db')
    cursor = conn.cursor()

    cursor.execute("SELECT TossWinner, TossDecision, Team1, Team2, Team1Players, Team2Players FROM ipl_match_list WHERE ID = ?;", (ID,))
    toss_info = cursor.fetchone()
    
    if toss_info:
        TossWinner, TossDecision, Team1, Team2, Team1Players_str, Team2Players_str = toss_info
        
        Team1Players = ast.literal_eval(Team1Players_str)
        Team2Players = ast.literal_eval(Team2Players_str)

        batting_team_players = Team1Players if TossWinner == Team1 else Team2Players

        if innings == 1:
            if TossDecision == "bat":
                remaining_players_list = [player for player in batting_team_players if player not in batting_list]
            else:
                remaining_players_list = [player for player in batting_team_players]
        else:  # innings == 2
            if TossDecision == "bat":
                remaining_players_list = [player for player in batting_team_players]
            else:
                remaining_players_list = [player for player in batting_team_players if player not in batting_list]

        conn.close()
        return remaining_players_list
    else:
        conn.close()
        return []

ID = 1312199
innings = 1
batters_list = data_manipulation.get_batting_positions_id_innings(ID, innings)
remaining_players(ID, innings, batters_list)


['YBK Jaiswal',
 'JC Buttler',
 'SV Samson',
 'D Padikkal',
 'SO Hetmyer',
 'R Parag',
 'R Ashwin',
 'TA Boult',
 'YS Chahal',
 'M Prasidh Krishna',
 'OC McCoy']

In [1]:
import search_results as sr
search_dict = sr.extract_elements()

In [2]:
search_dict

{'Players': ['JC Buttler',
  'R Vinay Kumar',
  'AC Blizzard',
  'RS Gavaskar',
  'Mohammed Siraj',
  'SP Narine',
  'AD Russell',
  'MS Dhoni',
  'SMSM Senanayake',
  'Arshdeep Singh',
  'M Prasidh Krishna',
  'GHS Garton',
  'BB Samantray',
  'Ramandeep Singh',
  'KMA Paul',
  'OF Smith',
  'FY Fazal',
  'MK Pandey',
  'DE Bollinger',
  'DP Nannes',
  'M Kaif',
  'HV Patel',
  'UT Khawaja',
  'DJ Harris',
  'AS Yadav',
  'MA Starc',
  'JP Behrendorff',
  'MS Wade',
  'BW Hilfenhaus',
  'T Shamsi',
  'KP Appanna',
  'S Dhawan',
  'JR Philippe',
  'Swapnil Singh',
  'Washington Sundar',
  'NLTC Perera',
  'SE Bond',
  'CA Pujara',
  'S Midhun',
  'Lalit Yadav',
  'JR Hazlewood',
  'KH Pandya',
  'Mohammad Nabi',
  'DJ Hussey',
  'D Brevis',
  'Jaskaran Singh',
  'Yashpal Singh',
  'P Kumar',
  'MS Bisla',
  'HE van der Dussen',
  'MN Samuels',
  'AS Roy',
  'J Suchith',
  'O Thomas',
  'DS Kulkarni',
  'S Badree',
  'VRV Singh',
  'DG Nalkande',
  'LJ Wright',
  'KS Bharat',
  'P Dharm

In [3]:
players = search_dict["Players"]
print(players)

['JC Buttler', 'R Vinay Kumar', 'AC Blizzard', 'RS Gavaskar', 'Mohammed Siraj', 'SP Narine', 'AD Russell', 'MS Dhoni', 'SMSM Senanayake', 'Arshdeep Singh', 'M Prasidh Krishna', 'GHS Garton', 'BB Samantray', 'Ramandeep Singh', 'KMA Paul', 'OF Smith', 'FY Fazal', 'MK Pandey', 'DE Bollinger', 'DP Nannes', 'M Kaif', 'HV Patel', 'UT Khawaja', 'DJ Harris', 'AS Yadav', 'MA Starc', 'JP Behrendorff', 'MS Wade', 'BW Hilfenhaus', 'T Shamsi', 'KP Appanna', 'S Dhawan', 'JR Philippe', 'Swapnil Singh', 'Washington Sundar', 'NLTC Perera', 'SE Bond', 'CA Pujara', 'S Midhun', 'Lalit Yadav', 'JR Hazlewood', 'KH Pandya', 'Mohammad Nabi', 'DJ Hussey', 'D Brevis', 'Jaskaran Singh', 'Yashpal Singh', 'P Kumar', 'MS Bisla', 'HE van der Dussen', 'MN Samuels', 'AS Roy', 'J Suchith', 'O Thomas', 'DS Kulkarni', 'S Badree', 'VRV Singh', 'DG Nalkande', 'LJ Wright', 'KS Bharat', 'P Dharmani', 'Misbah-ul-Haq', 'AK Markram', 'JM Sharma', 'HM Amla', 'RR Powar', 'KA Pollard', 'A Choudhary', 'Shahid Afridi', 'S Nadeem', '

In [4]:
players[:2]

['JC Buttler', 'R Vinay Kumar']

In [12]:
import data_manipulation
import pandas as pd
player_name = players[0]
c = 1
first_match_date, last_match_date = data_manipulation.get_first_and_last_match_dates(player_name)
combined_df_batting = pd.DataFrame(data_manipulation.batter_stats(player_name, first_match_date, last_match_date))
combined_df_bowling = pd.DataFrame(data_manipulation.bowler_stats(player_name, first_match_date, last_match_date))
print("Epoch = ",c,"/",len(players),"----",player_name)
for player_name in players[1:]:
    c+=1
    print("Epoch = ",c,"/",len(players),"----",player_name)
    first_match_date, last_match_date = data_manipulation.get_first_and_last_match_dates(player_name)
    batter_stats = data_manipulation.batter_stats(player_name, first_match_date, last_match_date)
    combined_df_batting = pd.concat([combined_df_batting, pd.DataFrame(batter_stats)], ignore_index=True)
    bowler_stats = data_manipulation.bowler_stats(player_name, first_match_date, last_match_date)
    combined_df_bowling = pd.concat([combined_df_bowling, pd.DataFrame(bowler_stats)], ignore_index=True)

Epoch =  1 / 606 ---- JC Buttler
Epoch =  2 / 606 ---- R Vinay Kumar
Epoch =  3 / 606 ---- AC Blizzard
Epoch =  4 / 606 ---- RS Gavaskar
Epoch =  5 / 606 ---- Mohammed Siraj
Epoch =  6 / 606 ---- SP Narine
Epoch =  7 / 606 ---- AD Russell
Epoch =  8 / 606 ---- MS Dhoni
Epoch =  9 / 606 ---- SMSM Senanayake
Epoch =  10 / 606 ---- Arshdeep Singh
Epoch =  11 / 606 ---- M Prasidh Krishna
Epoch =  12 / 606 ---- GHS Garton
Epoch =  13 / 606 ---- BB Samantray
Epoch =  14 / 606 ---- Ramandeep Singh
Epoch =  15 / 606 ---- KMA Paul
Epoch =  16 / 606 ---- OF Smith
Epoch =  17 / 606 ---- FY Fazal
Epoch =  18 / 606 ---- MK Pandey
Epoch =  19 / 606 ---- DE Bollinger
Epoch =  20 / 606 ---- DP Nannes
Epoch =  21 / 606 ---- M Kaif
Epoch =  22 / 606 ---- HV Patel
Epoch =  23 / 606 ---- UT Khawaja
Epoch =  24 / 606 ---- DJ Harris
Epoch =  25 / 606 ---- AS Yadav
Epoch =  26 / 606 ---- MA Starc
Epoch =  27 / 606 ---- JP Behrendorff
Epoch =  28 / 606 ---- MS Wade
Epoch =  29 / 606 ---- BW Hilfenhaus
Epoch =

In [13]:
combined_df_batting

,Name,Matches,Innings,Runs,Balls,Average,Strike Rate,Not Outs
0,JC Buttler,82,81,2832,1893,39.33,149.60,9
1,R Vinay Kumar,104,46,310,274,11.07,113.14,18
2,AC Blizzard,7,7,120,90,17.14,133.33,0
3,RS Gavaskar,2,1,2,8,2.00,25.00,0
4,Mohammed Siraj,65,20,96,109,13.71,88.07,13
...,...,...,...,...,...,...,...,...
601,RR Sarwan,4,4,73,75,18.25,97.33,0
602,RV Patel,4,3,31,30,15.50,103.33,1
603,AL Menaria,29,23,401,356,19.10,112.64,2
604,JL Denly,1,1,0,1,0.00,0.00,0


In [14]:
combined_df_bowling

,Name,Matches,Innings,Runs,Wickets,Economy,Balls,Strike Rate,Average
0,JC Buttler,82,0,0,0,NaN,0,NaN,NaN
1,R Vinay Kumar,104,104,2986,105,8.42,2127,20.26,28.44
2,AC Blizzard,7,0,0,0,NaN,0,NaN,NaN
3,RS Gavaskar,2,1,8,0,8.00,6,NaN,NaN
4,Mohammed Siraj,65,65,1951,60,8.78,1334,22.23,32.52
...,...,...,...,...,...,...,...,...,...
601,RR Sarwan,4,0,0,0,NaN,0,NaN,NaN
602,RV Patel,4,1,22,0,7.33,18,NaN,NaN
603,AL Menaria,29,12,144,3,7.85,110,36.67,48.00
604,JL Denly,1,0,0,0,NaN,0,NaN,NaN


In [15]:
import sqlite3


conn = sqlite3.connect('./ipl_database.db')

# Write the DataFrame to a SQL database
combined_df_batting.to_sql('ipl_batter_list', conn, index=False, if_exists='replace')
combined_df_bowling.to_sql('ipl_bowler_list', conn, index=False, if_exists='replace')

# Close the connection
conn.close()

In [17]:
import sqlite3
import pandas as pd

# Connect to the database
conn = sqlite3.connect('./ipl_database.db')

# Read the combined_df_batting table from the database
combined_df_batting = pd.read_sql_query("SELECT * FROM ipl_batter_list WHERE Balls >= 60 ORDER BY Average DESC", conn)

# Close the connection
conn.close()

# Display the DataFrame
print(combined_df_batting)


              Name  Matches  Innings  Runs  Balls  Average  Strike Rate   
0       MN van Wyk        5        5   167    132    55.67       126.52  \
1         KL Rahul      109      100  3895   2861    46.93       136.14   
2         AC Voges        9        7   181    143    45.25       126.57   
3          HM Amla       16       16   577    407    44.38       141.77   
4    Iqbal Abdulla       49       13    88     84    44.00       104.76   
..             ...      ...      ...   ...    ...      ...          ...   
314     SL Malinga      122       25    88     99     5.50        88.89   
315      YS Chahal      131       20    37     86     5.29        43.02   
316       R Sharma       60       20    66     75     4.71        88.00   
317       RP Singh       82       30    52     76     3.47        68.42   
318       S Nadeem       72       22    39     87     2.79        44.83   

     Not Outs  
0           2  
1          17  
2           3  
3           3  
4          11  
.. 

In [51]:
import sqlite3
import pandas as pd

def innings_by_innings_list_match_list_details(id_list):
    Date = []
    Venue = []
    Season = []
    MatchNumber = []
    City = []
    conn = sqlite3.connect('./ipl_database.db')
    cursor = conn.cursor()
    for ID in id_list:
        query = '''
        SELECT 
            Date, Venue, Season, MatchNumber, City   
        FROM 
            ipl_match_list
        WHERE 
            ID = ?
        GROUP BY 
            ID;
        '''
        cursor.execute(query, (ID,))
        x = cursor.fetchall()[0]
        Date.append(x[0])
        Venue.append(x[1])
        Season.append(x[2])
        MatchNumber.append(x[3])
        City.append(x[4])
        
   

    conn.close()

    return Date, Venue, Season, MatchNumber, City

def innings_by_innings_list_batting(name, id_list):
    batter,	Runs_Scored, Balls_Faced, Four_Count, Six_Count, Out, Dismissal_Type, fielders_involved, Bowler = [], [], [], [], [], [], [], [], []

    batter_dict = {}
    conn = sqlite3.connect('./ipl_database.db')
    cursor = conn.cursor()
    for ID in id_list:
        query = '''
        SELECT 
            batter AS Name,
            SUM(batsman_run) AS Runs_Scored,
            COUNT(CASE WHEN extra_type IS NULL OR extra_type = 'byes' OR extra_type = 'legbyes' OR extra_type = 'noballs' THEN 1 ELSE NULL END) AS Balls_Faced,
            SUM(CASE WHEN batsman_run = 4 THEN 1 ELSE 0 END) AS Four_Count,
            SUM(CASE WHEN batsman_run = 6 THEN 1 ELSE 0 END) AS Six_Count,
            COUNT(CASE WHEN isWicketDelivery = 1 THEN 1 ELSE NULL END) AS Out,
            MAX(CASE WHEN isWicketDelivery = 1 THEN kind ELSE NULL END) AS Dismissal_Type,
            MAX(CASE WHEN isWicketDelivery = 1 THEN fielders_involved ELSE NULL END) AS fielders_involved,
            MAX(CASE WHEN isWicketDelivery = 1 AND kind <> 'run out' THEN bowler ELSE NULL END) AS Bowler
            
        FROM 
            ipl_ball_by_ball
        WHERE 
            batter = ? AND ID = ?
        GROUP BY 
            batter;
        '''
        cursor.execute(query, (name, ID))
        x = cursor.fetchall()[0]
        if x:
            batter.append(x[0])
            Runs_Scored.append(x[1])
            Balls_Faced.append(x[2])
            Four_Count.append(x[3])
            Six_Count.append(x[4])
            Out.append(x[5])
            Dismissal_Type.append(x[6])
            fielders_involved.append(x[7])
            Bowler.append(x[8])
        else:
            batter.append(None)
            Runs_Scored.append(None)
            Balls_Faced.append(None)
            Four_Count.append(None)
            Six_Count.append(None)
            Out.append(None)
            Dismissal_Type.append(None)
            fielders_involved.append(None)
            Bowler.append(None)
    
    Date, Venue, Season, MatchNumber, City = innings_by_innings_list_match_list_details(id_list)
    batter_dict["Date"] = Date
    batter_dict["MatchNumber"] = MatchNumber
    batter_dict["Season"] = Season
    batter_dict["batter"] = batter
    batter_dict["Runs_Scored"] = Runs_Scored
    batter_dict["Balls_Faced"] = Balls_Faced
    batter_dict["Four_Count"] = Four_Count
    batter_dict["Six_Count"] = Six_Count
    batter_dict["Out"] = Out
    batter_dict["Dismissal_Type"] = Dismissal_Type
    batter_dict["fielders_involved"] = fielders_involved
    batter_dict["Bowler"] = Bowler
    batter_dict["Venue"] = Venue
    batter_dict["City"] = City
    batter_dict["ID"] = id_list
    
    conn.close()
    df = pd.DataFrame(batter_dict)
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values(by='Date', ascending=True)
    return df

innings_by_innings_list_batting("Shubman Gill", [1254117, 1304103])

,Date,MatchNumber,Season,batter,Runs_Scored,Balls_Faced,Four_Count,Six_Count,Out,Dismissal_Type,fielders_involved,Bowler,Venue,City,ID
0,2021-10-15,Final,2021,Shubman Gill,51,43,6,0,1,lbw,None,DL Chahar,Dubai International Cricket Stadium,Dubai,1254117
1,2022-05-10,57,2022,Shubman Gill,63,49,7,0,0,None,None,None,Maharashtra Cricket Association Stadium,Pune,1304103


In [1]:
id_list = [1216536, 1216533, 1216509, 1216528, 1216525, 1216501, 1216513, 1216516, 1216539, 1216496, 1216492, 1181768, 1181767, 1181764, 1178430, 1178425, 1178419, 1178416, 1178414, 1178408, 1178404, 1178400, 1178398, 1178393, 1175370, 1175367, 1175360, 1175356, 1136620, 1136617, 1136612, 1136606, 1136603, 1136595, 1136593, 1136590, 1136587, 1136584, 1136580, 1136577, 1136572, 1136565, 1136561, 1082646, 1082633, 1082628, 1082610, 1082607, 1082598, 1082595, 1082591, 981019, 981013, 981011, 980999, 980995, 980987, 980981, 980977, 980969, 980959, 980953, 980937, 980931, 980927, 980921, 980907, 829819, 829811, 829797, 829787, 829777, 829769, 829763, 829749, 829741, 829735, 734041, 734033, 734017, 734003, 733999, 733989, 733979, 733975, 729315, 729305, 729297, 729291, 729285, 598072, 598071, 598065, 598063, 598058, 598052, 598049, 598047, 598043, 598036, 598032, 598026, 598023, 598019, 598014, 598005, 548377, 548373, 548365, 548361, 548358, 548354, 548349, 501263, 501258, 501252, 501249, 501240, 501235, 501231, 501225, 501220, 501214, 501209, 419158, 419154, 419150, 419143, 419141, 419137, 336040, 336038, 336019, 336036, 336027, 336023, 336015, 336011, 336008, 336005, 336000, 335992, 335990, 335987, 335984]

In [2]:
team1_list = ['Royal Challengers Bangalore', 'Rajasthan Royals', 'Chennai Super Kings']
team2_list = ['Sunrisers Hyderabad', 'Rising Pune Supergiant', 'Rajasthan Royals', 'Delhi Daredevils', 'Rising Pune Supergiants', 'Delhi Capitals', 'Gujarat Lions', 'Kings XI Punjab', 'Kochi Tuskers Kerala', 'Deccan Chargers', 'Chennai Super Kings', 'Kolkata Knight Riders', 'Mumbai Indians', 'Pune Warriors', 'Royal Challengers Bangalore']

In [3]:
name = "SR Watson"

In [4]:
venue = ["M.Chinnaswamy Stadium", "Sawai Mansingh Stadium", "MA Chidambaram Stadium"]

In [13]:
import sqlite3
import pandas as pd
def select_matches_home(player_name, id_list, teamfor_list, teamagainst_list, venue):

    # Connect to the database
    conn = sqlite3.connect('./ipl_database.db')
    
    # Format teamfor_list_str for the query
    teamfor_list_str = ', '.join([f'"{team}"' for team in teamfor_list])

    # Format teamagainst_list_str for the query
    teamagainst_list_str = ', '.join([f'"{team}"' for team in teamagainst_list])

    # Format id_list_str for the query
    id_list_str = ', '.join([str(id) for id in id_list])
    print(type(teamfor_list_str))
    print(type(teamagainst_list_str))
    print(type(id_list_str))   

    # SQL query
    query = f'''
    SELECT m.ID, 
    CASE 
        WHEN p1 LIKE '%' || ? || '%' THEN Team1 
        WHEN p2 LIKE '%' || ? || '%' THEN Team2 
    END AS PlayerTeam,
    CASE 
        WHEN p1 LIKE '%' || ? || '%' THEN Team2 
        WHEN p2 LIKE '%' || ? || '%' THEN Team1 
    END AS OpponentTeam,
    v.City, 
    m.Date
    FROM ipl_match_list m
    JOIN ipl_venue_list v ON m.Venue = v.Venue
    CROSS JOIN (SELECT ? AS p1, ? AS p2) AS params
    WHERE (Team1Players LIKE '%' || ? || '%' OR Team2Players LIKE '%' || ? || '%')
    AND v.Teams LIKE '%' || ? || '%';
    '''

    # Fetch results into DataFrame
    df = pd.read_sql_query(query, conn, params=('%' + player_name + '%', '%' + player_name + '%', '%' + player_name + '%', '%' + player_name + '%', '%' + player_name + '%', teamfor_list_str, teamagainst_list_str, id_list_str, '%' + venue + '%'))

    # Close the connection
    conn.close()

    return df

In [14]:
select_matches_home(name, id_list, team1_list, team2_list, venue)

<class 'str'>
<class 'str'>
<class 'str'>


TypeError: can only concatenate str (not "list") to str

In [18]:
import sqlite3
import pandas as pd
def select_matches_away(player_name, id_list, teamfor_list, teamagainst_list, venue):

    # Connect to the database
    conn = sqlite3.connect('./ipl_database.db')
    cursor = conn.cursor()

    # Format teamfor_list for the query
    teamfor_list_str = ', '.join([f'"{team}"' for team in teamfor_list])
    teamagainst_list_str = ', '.join([f'"{team}"' for team in teamagainst_list])
    id_list_str = ', '.join([str(id) for id in id_list])
    # SQL query
    query = f'''
    SELECT m.ID, m.Venue, 
            CASE WHEN m.Team1Players NOT LIKE ? THEN m.Team1 ELSE m.Team2 END AS Team
    FROM ipl_match_list m
    JOIN ipl_venue_list v ON m.Venue = v.Venue
    WHERE (m.Team1Players NOT LIKE ? 
    AND m.Team1 IN ({teamagainst_list_str})
    AND v.Teams LIKE '%' || m.Team1 || '%')
    OR
    (m.Team2Players NOT LIKE ? 
    AND m.Team2 IN ({teamagainst_list_str})
    AND v.Teams LIKE '%' || m.Team2 || '%')
    AND m.ID IN ({id_list_str})
    '''

    # Fetch results into DataFrame
    df = pd.read_sql_query(query, conn, params=('%' + player_name + '%', '%' + player_name + '%'))

    # Close the connection
    conn.close()

    return df

In [19]:
select_matches_away(name, id_list, team1_list, team2_list, venue)

DatabaseError: Execution failed on sql '
    SELECT m.ID, m.Venue, 
            CASE WHEN m.Team1Players NOT LIKE ? THEN m.Team1 ELSE m.Team2 END AS Team
    FROM ipl_match_list m
    JOIN ipl_venue_list v ON m.Venue = v.Venue
    WHERE (m.Team1Players NOT LIKE ? 
    AND m.Team1 IN ("Sunrisers Hyderabad", "Rising Pune Supergiant", "Rajasthan Royals", "Delhi Daredevils", "Rising Pune Supergiants", "Delhi Capitals", "Gujarat Lions", "Kings XI Punjab", "Kochi Tuskers Kerala", "Deccan Chargers", "Chennai Super Kings", "Kolkata Knight Riders", "Mumbai Indians", "Pune Warriors", "Royal Challengers Bangalore")
    AND v.Teams LIKE '%' || m.Team1 || '%')
    OR
    (m.Team2Players NOT LIKE ? 
    AND m.Team2 IN ("Sunrisers Hyderabad", "Rising Pune Supergiant", "Rajasthan Royals", "Delhi Daredevils", "Rising Pune Supergiants", "Delhi Capitals", "Gujarat Lions", "Kings XI Punjab", "Kochi Tuskers Kerala", "Deccan Chargers", "Chennai Super Kings", "Kolkata Knight Riders", "Mumbai Indians", "Pune 